In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from explorers.PPO_explorer import PPO_explorer
from explorers.DynaPPO_explorer import DynaPPO_explorer
from explorers.CMAES_explorer import CMAES_explorer
from evaluators.Evaluator import Evaluator

from utils.sequence_utils import generate_random_mutant, generate_random_sequences

## 1. Building oracles for landscapes

In [ ]:
from models.Ground_truth_oracles.RNA_landscape_models import RNA_landscape_constructor
from models.Ground_truth_oracles.TF_binding_landscape_models import *

In [ ]:
LANDSCAPE_TYPES ={"RNA": [0, 1], 'TF':['SIX6_REF_R1']}

landscape_constructor=TF_binding_landscape_constructor()
landscape_constructor.load_landscapes(landscapes_to_test = ['SIX6_REF_R1'])

landscape_generator = landscape_constructor.generate_from_loaded_landscapes()

#take one landscape oracle from the generator
landscape_object=next(landscape_generator)

print(landscape_object['landscape_id'])
print(landscape_object['starting_seqs'])
landscape_oracle = landscape_object['landscape_oracle']

wt = landscape_object['starting_seqs']['TF0']

## 2. Building a model of the landscape

In [ ]:
from utils.model_architectures import CNNa
from models.Noisy_models.Neural_network_models import NN_model

alphabet = "ATGC"

training_sequences = list(set([generate_random_mutant(wt, 0.2, alphabet) for i in range(0, 100)]))
cnn_architecture=CNNa(len(wt), alphabet=alphabet)
noisy_nn_model=NN_model(landscape_oracle, cnn_architecture)
noisy_nn_model.reset()
noisy_nn_model.update_model(training_sequences)

## 3. Exploring the landscape with a model

### CMA-ES explorer

In [ ]:
cmaes_explorer = CMAES_explorer(alphabet=alphabet)
cmaes_explorer.debug = True
cmaes_explorer.set_model(noisy_nn_model)
cmaes_explorer.initialize_params()

In [ ]:
cmaes_explorer.propose_samples()

In [ ]:
num_of_rounds = 5
cmaes_explorer.run(num_of_rounds, verbose=True)

### DyNA-PPO explorer

In [ ]:
from models.Noisy_models.Ensemble import Ensemble_models

ensemble_model = Ensemble_models(list_of_models=[NN_model(landscape_oracle, cnn_architecture),
                                                NN_model(landscape_oracle, cnn_architecture),
                                                NN_model(landscape_oracle, cnn_architecture)])
ensemble_model.reset()
ensemble_model.update_model(training_sequences)

dynappo_explorer = DynaPPO_explorer(
    alphabet=alphabet,
    oracle=landscape_oracle,
    num_experiment_rounds=1,
    num_model_rounds=1)
dynappo_explorer.debug = True
dynappo_explorer.set_model(ensemble_model)

dynappo_explorer.learn_policy()

In [ ]:
num_of_rounds = 5
dynappo_explorer.run(num_of_rounds, verbose=True)

### PPO explorer

In [ ]:
ppo_explorer = PPO_explorer(
    alphabet=alphabet)
ppo_explorer.debug = True
ppo_explorer.set_model(noisy_nn_model)

In [ ]:
ppo_explorer.pretrain_agent()

In [ ]:
noisy_nn_model=NN_model(landscape_oracle, cnn_architecture)
noisy_nn_model.reset()
noisy_nn_model.update_model(training_sequences)

num_of_rounds = 5
ppo_explorer.run(num_of_rounds, verbose=True)